In [1]:
import pandas as pd
import os
from random import sample
from pathlib import Path
from tqdm.auto import tqdm
import sys
sys.path.append("../")

In [2]:
N_files_to_sample = 20
READ_FILE = True
PICKLE_PATH = Path('test.pkl.gz')

if READ_FILE:
    df = pd.read_pickle(PICKLE_PATH)
else:
    data_path = Path("../data/simulations/psuu_run_20240325T223806Z/")
    pickle_files: list[str] = [f for f in os.listdir(data_path) if ".pkl" in f]

    files_to_open = [data_path / f for f in sample(pickle_files, N_files_to_sample)]
    dfs = []
    for f in tqdm(files_to_open):
        dfs.append(pd.read_pickle(f))

    df = pd.concat(dfs)
    del dfs
    df.to_pickle(PICKLE_PATH)

In [3]:
import aztec_gddt.metrics as m

In [7]:
governance_surface_params = [
    'phase_duration_proposal_min_blocks',
    'phase_duration_proposal_max_blocks',
    'phase_duration_reveal_min_blocks',
    'phase_duration_reveal_max_blocks',
    'phase_duration_commit_bond_min_blocks',
    'phase_duration_commit_bond_max_blocks',
    'phase_duration_rollup_min_blocks',
    'phase_duration_rollup_max_blocks',
    'phase_duration_race_min_blocks',
    'phase_duration_race_max_blocks',
    'reward_per_block'
]
trajectory_id_columns = ['simulation', 'subset', 'run']
agg_columns = ['simulation', 'subset', 'run'] + governance_surface_params

In [9]:
KPIs_per_trajectory = {
   # 'K1': m.find_proportion_race_mode,
    'K2': m.find_proportion_slashed_due_to_prover
}

kpi_dfs = []
for kpi, kpi_f in KPIs_per_trajectory.items():
    kpi_s = df.groupby(agg_columns).apply(kpi_f, include_groups=False)
    kpi_s.name = kpi
    kpi_dfs.append(kpi_s)


KeyError: 'process_id'